In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys 
from torch.nn import functional as F
import configparser
import pandas as pd 

import numpy as np
import torch
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
from os.path import join as oj
sys.path.insert(0, "../src")
from unet import UNet
import my_data
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
config = configparser.ConfigParser()
config.read('../config.ini');
data_path =config['DATASET']['data_path'] 
fig_path =config['PATHS']['figure_path'] 
import segmentation_models_pytorch as smp


In [ ]:
# need exp with no active learning
# need exp with prop increasingf
fnames = sorted([oj(config['PATHS']['model_path'] , fname) for fname in os.listdir(config['PATHS']['model_path'] ) if "pkl" in fname]) 
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects()
# results = results[results['experiment_name'] == 'ActiveLearningGr2']
# results = results[results['add_ratio'] == 0.12]

results.reset_index(inplace=True)
results[[ 'cost_function', 'add_ratio',  'test_dice_score','test_dice_score_all', 
         'experiment_name',]].groupby([ 'experiment_name', 'cost_function','add_ratio', 
     ]).mean()



In [ ]:
import seaborn as sns
#lineplot
results_here = results[results.experiment_name == 'ActiveLearningGr2']


results_here_copy = results[results.experiment_name == 'GraphiteNoAq']
results_here_copy.cost_function = 'cut_off_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'GraphiteNoAq']
results_here_copy.cost_function = 'random_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'GraphiteNoAq']
results_here_copy.cost_function = 'emc'
results_here = pd.concat([results_here, results_here_copy])



plt.figure(figsize=(20,10))
#rename 
results_here.cost_function = results_here.cost_function.replace('cut_off_cost', 'Uncertainty')
results_here.cost_function = results_here.cost_function.replace('random_cost', 'Random')

results_here.cost_function = results_here.cost_function.replace('emc', 'EMC')

plt.figure(figsize=(10,5))

#rename column
results_here = results_here.rename(columns={'test_dice_score_all': 'Dice Score'})
results_here.add_ratio = results_here.add_ratio*100
results_here = results_here.rename (columns={'add_ratio': 'Percentage of data'})

results_here = results_here.rename(columns={'cost_function': 'Cost function'})

sns.set(font_scale=2)
sns.set_style("whitegrid")

g_results = sns.lineplot(data=results_here, x="Percentage of data", y="Dice Score", hue="Cost function", errorbar='se', palette="tab10", linewidth=2.5)

plt.ylim(.1, .85)
line_y = 0.743
plt.axhline(line_y, color='k', linestyle='--', label='Full dataset')
plt.axhspan(line_y - 0.047, line_y + 0.047, color='gray', alpha=0.3)
# an arrow that points to the h line and says "Full dataset"
plt.annotate('Full dataset', xy=(0, line_y), xytext=(0, line_y +.1), arrowprops=dict(facecolor='black', shrink=0.1),)
# savefig
plt.savefig(oj(fig_path, 'results_lineplot_graphite_all.pdf'), dpi=300, bbox_inches='tight')